# rUBot SLAM

Using SLAM (short for Simultaneous Localization and Mapping) techniques, you will be able to execute autonomous navigation with our rUBot.

SLAM is a technique used in robotics to explore and map an unknown environment while estimating the pose of the robot itself. As it moves all around, it will be acquiring structured information of the surroundings by processing the raw data coming from its sensors.

For optimal and easy-to-understand coverage of the topic of SLAM, we will implement a 360º-coverage Laser Distance Sensor (LDS) in the virtual robot.

There are low-cost versions of this sensor technology, such as EAI YDLIDAR X4 (available at https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html), which is the one we will make use on our rUBot.

### Install ROS navigation & SLAM packages
First, let's prepare your machine with the required ROS packages needed for the navigation stack (http://wiki.ros.org/navigation):

In [ ]:
sudo apt-get install ros-melodic-navigation

And finally the slam_gmapping package, that is already available in its binary version (https://wiki.ros.org/slam_gmapping):

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

Open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/rUBotCoop_LabProject/devel/setup.bash

export ROS_IP=192.168.18.83

export ROS_MASTER_URI=http://192.168.18.83:11311

### What do you need to perform robot navigation with ROS?
- Mapping: First, you need a map
- Localization: Next you need to localize the robot on that map
- Path Planning: Now you can send goal locations to the robot
- Obstacle avoidance: Finally, you need to avoid obstacles

First of all we create a "mecanum_slam" package with dependencies: roscpp std_msgs sensor_msgs geometry_msgs nav_msgs tf gazebo_ros

In [ ]:
catkin_create_pkg rubotcoop_slam rospy std_msgs sensor_msgs geometry_msgs nav_msgs

Let's follow these steps to build the map of a simple Gazebo world called square.world:

- Launch the robot model within a modeled environment by running the following line of code:

In [ ]:
roslaunch rubotcoop_slam rubot_world.launch world:=world1.world

- Launch the SLAM mapping ROS package, including an RViz visualization that superimposes the virtual model of the robot with the actual scan data:

In [ ]:
roslaunch rubotcoop_slam rubot_slam.launch

Teleoperate the robot to make it cover as much as possible of the surface of the current Gazebo world. Let's do this as usual with the teleoperation package:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
or
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

<img src="./Images/4_rubot_slam_gmapping.png">

As you move the robot, the LDS sensor will acquire scan data from the unknown areas, and you will receive feedback in the RViz window.

- Once you've finished the exploration, save the map, generating two files of the formats indicated in the preceding SLAM process subsection, that is, .pgm and .yaml

In [ ]:
rosrun map_server map_saver -f /media/sf_github_manelpuig/rUBot_mecanum_ws/src/robot_slam/nexus_slam/maps/map_world1

You will get two files in the root folder of your workspace: map_stage_2.pgm and map_stage_2.yaml.

Provided with the map, we are ready to perform robot navigation with the rubot

### Driving along a planned trajectory using navigation
First, close the "rubot_slam.launch" terminal. Then, as in the SLAM process, let's proceed step by step to perform some navigation:
- Set up the navigation algorithm and launch RViz. We will use AMCL, the most common choice for effective navigation.

In this step, we also provide the costmap that the robot built before. To do this, you just have to reference the .yaml map file you created before. Make sure that the corresponding .pgm file has the same name and is placed in the same location.

In [ ]:
roslaunch nexus_slam rubot_navigation.launch map_file:=/media/sf_github_manelpuig/rUBot_mecanum_ws/src/robot_slam/nexus_slam/maps/map_world1.yaml

- The RViz window, shown in the following screenshot, lets you visualize the robot in the environment and mark the target pose (position and orientation) that it should achieve: First of all, you have to tell the robot that this is the initial pose by pressing the 2D Pose Estimate button. Then, mark it on screen (in this particular case, it isn't necessary, since the initial pose is the same as the one the robot had when it started to build the map)

Afterward, you can press 2D Nav Goal button and set the target to the bottom-left corner by clicking the left mouse button. Release the mouse when the arrow has the desired orientation. After releasing, the robot will compute the path to follow and start navigating autonomously.

The orientation of the red arrow tells the GoPiGo3 in what direction it should stay facing once it has arrived at the target, and the curved line going from the robot to the target is the planned path. Since it has a map of the environment available, the robot is able to plan a path that avoids the obstacles.

The blue square around the robot represents the local window for obstacle avoidance planning. This is used by the Dynamic Window Approach (DWA) method, which generates a local path that efficiently evades the obstacles. The DWA method performs the calculations taking into account the robot's dynamics, in particular, its limited velocity and acceleration.

<img src="./Images/4_nexus_slam1.png">

<img src="./Images/4_nexus_slam2.png">